In [198]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz
import scipy.stats


import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

# Team Data, including standings

In [199]:
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams'

teams = requests.get(teamURL).json()
teams = teams['teams']

team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['abbreviation']
    name = team['name']
    id = team['id']

    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name], 'id': [id]})

    team_names = pd.concat([team_names, temp], ignore_index=True)
    
# ----------------------------------------------------------------------#    
    
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams/'
appendix = '/stats'

team_stats = pd.DataFrame()

for index, row in team_names.iterrows():

  fetchURL = teamURL + str(row['id']) + appendix

  # print(fetchURL)

  team = requests.get(fetchURL).json()
  stats = team['stats'][0]['splits'][0]['stat']
  name = team['stats'][0]['splits'][0]['team']['name']

  temp = pd.DataFrame([stats])
  temp['name'] = name

  team_stats = pd.concat([team_stats, temp], ignore_index=True)


team_stats = pd.merge(team_stats, team_names, on='name')
team_stats.set_index('name', inplace=True)

# Web Scrape Sked Data

### Set Date for one before and one after window

In [200]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/schedule'

# Date of lookahead
start = '?startDate=2023-11-02'

end = '&endDate=2023-11-06'

skedURL = baseURL + start + end

sked = requests.get(skedURL).json()

In [201]:
days = sked['dates']

eastern = pytz.timezone('US/Eastern')

In [202]:
# See teams with most games

teams_data = {}
games_data = []

for day in days[1:4]:
# for day in days:
    games = day['games']

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']

        if home_team not in teams_data:
            teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
        if away_team not in teams_data:
            teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

        teams_data[home_team]['played'] += 1
        teams_data[away_team]['played'] += 1
        teams_data[home_team]['home'] += 1
        teams_data[away_team]['away'] += 1

        games_data.append(game['gamePk'])

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

teams_df

,played,home,away
St. Louis Blues,2,2,0
Chicago Blackhawks,2,2,0
Buffalo Sabres,2,1,1
Philadelphia Flyers,2,1,1
Vegas Golden Knights,2,1,1
New Jersey Devils,2,0,2
Edmonton Oilers,1,1,0
Arizona Coyotes,1,1,0
Detroit Red Wings,1,1,0
Ottawa Senators,1,1,0


# Scrape the Box Score data for season to-date

In [203]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202302'
game = 1
range_end = int(str(games_data[0]-1)[-4:])

date_format_game = "%d-%m-%Y"
date_format = "%d-%m-%Y"
date_format_birth = "%Y-%m-%d"

awaylogs = {}
homelogs = {}

# Subtract games not done on Sunday!
for i in range(range_end):
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')
    game_date = datetime.strptime(formatted_date_string, date_format)

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player
        awaylogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name
            
                

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player
        homelogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days

https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020016/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2023020136/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020137/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020138/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020139/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020140/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020141/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020142/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020143/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020144/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020145/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020146/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020147/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020148/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020149/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020150/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020151/feed/live
https://

In [204]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_df = pd.concat([home_df, away_df], axis=0)

all_df['date'] = pd.to_datetime(all_df['date'], format='%d-%m-%Y')
all_df['birthdate'] = pd.to_datetime(all_df['birthdate'], format='%Y-%m-%d')
all_df = all_df.sort_values(by='date')

In [205]:
# 22-23 FANTASYPOINTS
for index, row in all_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
        all_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'decisionPoints'] = d
        all_df.at[index, 'wins'] = w

    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'specialTeams'] = st


In [206]:
# Remove all zero ice time games

all_df = all_df[all_df['timeOnIce'] != 0]

In [207]:
# Replace Sebastian Aho, D, with Sebastian Aho (D)

all_df.loc[(all_df['name'] == 'Sebastian Aho') & (all_df['position'] == 'D')] = all_df.loc[(all_df['name'] == 'Sebastian Aho') & (all_df['position'] == 'D')].replace('Sebastian Aho', 'Sebastian Aho (D)')

In [208]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5943 entries, ID8478519-10-10-2023 to ID8481535-02-11-2023
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   timeOnIce                   5943 non-null   int32         
 1   assists                     5943 non-null   int32         
 2   goals                       5943 non-null   int32         
 3   shots                       5943 non-null   int32         
 4   hits                        5943 non-null   int32         
 5   powerPlayGoals              5943 non-null   int32         
 6   powerPlayAssists            5943 non-null   int32         
 7   penaltyMinutes              5943 non-null   int32         
 8   faceOffPct                  2179 non-null   object        
 9   faceOffWins                 5943 non-null   int32         
 10  faceoffTaken                5943 non-null   int32         
 11  takeaways                 

# Build Summary Stats Season to Date

In [209]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats = all_df.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats['gamePlayed'] = summary_stats['gamePlayed'].astype(int)
summary_stats['decisionPoints'] = summary_stats['decisionPoints'].astype(int)
summary_stats['wins'] = summary_stats['wins'].astype(int)
summary_stats['specialTeams'] = summary_stats['specialTeams'].astype(int)


summary_stats['FPP60'] = (summary_stats['fantasyPoints'] / summary_stats['timeOnIce'] * 3600).round(2)
summary_stats['FPPG'] = (summary_stats['fantasyPoints'] / summary_stats['gamePlayed']).round(2)

summary_stats = summary_stats.sort_values(by='FPPG', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats.set_index('name', inplace=True)

player_list = summary_stats.index.to_list()

summary_stats.head()

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,...,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,
Semyon Varlamov,ID8473575,NYI,G,10747,3,0,0,0,108,0,...,85,4,88,16,8,2,2,29.0,9.71,9.67
Jeremy Swayman,ID8480280,BOS,G,18230,5,0,0,0,152,0,...,108,8,114,30,20,5,1,38.0,7.50,7.60
Anthony Stolarz,ID8476932,FLA,G,3569,1,0,0,0,28,0,...,23,2,23,3,4,1,0,7.4,7.46,7.40
Jonathan Quick,ID8471734,NYR,G,8752,3,0,0,0,57,0,...,54,1,55,1,8,2,1,20.2,8.31,6.73
Jake Oettinger,ID8479979,DAL,G,22174,6,0,0,0,198,0,...,145,5,156,37,21,5,0,34.2,5.55,5.70


In [210]:
skaters_summary = summary_stats.loc[summary_stats['position'] != "G"]
skaters_summary = skaters_summary.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary = summary_stats.loc[summary_stats['position'] == "G"]
goalies_summary = goalies_summary.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])

goalies = all_df.loc[all_df['position'] == 'G']

skaters = all_df.loc[all_df['position'] != 'G']
skaters['specialTeams'] = skaters['specialTeams'].astype(int)

# Pull in Rostership Data from ESPN

In [211]:
espnapi = 'https://lm-api-reads.fantasy.espn.com/apis/v3/games/fhl/seasons/2024/players?view=players_wl&filter=%7B%22filterActive%22%3A%7B%22value%22%3Atrue%7D%7D'

rosters = requests.get(espnapi).json()

# pprint(rosters[9])

rostership = pd.DataFrame()

for player in rosters:
    n = player['fullName']
    percent = player['ownership']['percentOwned']
    pos = player['eligibleSlots']
    dpos = player['defaultPositionId']
    if (n == 'Sebastian Aho') & (dpos == 4):
        n = 'Sebastian Aho (D)'
#     print(n)
    temp = pd.DataFrame({'name': [n], 'rostered': [percent]})
    rostership = pd.concat([rostership, temp], ignore_index=True)
    
rostership.head()

,name,rostered
0,Calle Jarnkrok,0.921484
1,Charlie Coyle,3.974375
2,Charlie Lindgren,0.196141
3,Mike Vecchione,0.022771
4,Olle Alsing,0.007591


### Fix the accented players

In [212]:
fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Alex Barre-Boulet': 'Alex Barré-Boulet',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
    'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker'
}

rostership['name'].replace(fixes, inplace=True)


In [213]:
#Merge

summary_stats = summary_stats.merge(rostership, on='name', how='left')

In [215]:
#Check for nulls
rows_with_null_rostered = summary_stats[summary_stats['rostered'].isna()]
rows_with_null_rostered

# summary_stats_with_percent.loc[rows_with_null_rostered.index, 'rostered'] = 0

In [217]:
summary_stats

,name,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,...,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,rostered
0,Semyon Varlamov,ID8473575,NYI,G,10747,3,0,0,0,108,...,4,88,16,8,2,2,29.0,9.71,9.67,3.297906
1,Jeremy Swayman,ID8480280,BOS,G,18230,5,0,0,0,152,...,8,114,30,20,5,1,38.0,7.50,7.60,91.198115
2,Anthony Stolarz,ID8476932,FLA,G,3569,1,0,0,0,28,...,2,23,3,4,1,0,7.4,7.46,7.40,0.421171
3,Jonathan Quick,ID8471734,NYR,G,8752,3,0,0,0,57,...,1,55,1,8,2,1,20.2,8.31,6.73,7.526228
4,Jake Oettinger,ID8479979,DAL,G,22174,6,0,0,0,198,...,5,156,37,21,5,0,34.2,5.55,5.70,99.606162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,Dan Vladar,ID8478435,CGY,G,7191,2,0,0,0,57,...,0,40,17,4,1,0,-4.4,-2.20,-2.20,0.307164
748,Cayden Primeau,ID8480051,MTL,G,3468,1,0,0,0,33,...,0,26,7,0,0,0,-2.2,-2.28,-2.20,0.066901
749,Pheonix Copley,ID8477831,LAK,G,7957,3,0,0,0,52,...,3,40,9,5,1,0,-8.8,-3.98,-2.93,4.066116
750,Pyotr Kochetkov,ID8481611,CAR,G,9151,3,0,0,0,67,...,3,52,12,0,0,0,-10.8,-4.25,-3.60,27.864191


# Check teams with back-to-backs

In [218]:
# See teams that play back-to-back

def check_consecutive_dates(game1, game2):
    date_format = '%Y-%m-%dT%H:%M:%SZ'
    date1 = datetime.strptime(game1['gameDate'], date_format)
    date2 = datetime.strptime(game2['gameDate'], date_format)
    return (date2 - date1).days == 1

teams_playing_consecutive_days = set()

for i in range(len(days)-1):
    current_day_games = days[i]['games']
    next_day_games = days[i+1]['games']

    teams_on_next_day = set()

    for game2 in next_day_games:
        home_team_2 = game2['teams']['home']['team']['name']
        away_team_2 = game2['teams']['away']['team']['name']
        teams_on_next_day.add(home_team_2)
        teams_on_next_day.add(away_team_2)

    for game1 in current_day_games:
        home_team_1 = game1['teams']['home']['team']['name']
        away_team_1 = game1['teams']['away']['team']['name']

        if (home_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(home_team_1)
        elif (away_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(away_team_1)

print("Teams playing on consecutive days:")
for team in teams_playing_consecutive_days:
    print(team)

Teams playing on consecutive days:
New Jersey Devils
St. Louis Blues
Chicago Blackhawks
Buffalo Sabres


# Check teams not playing

In [219]:
team_check = teams_df.index.to_list()

nhl_teams = [
    'New Jersey Devils', 'New York Islanders', 'New York Rangers',
    'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins',
    'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators',
    'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers',
    'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks',
    'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues',
    'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
    'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars',
    'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets',
    'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes',
    'Vegas Golden Knights', 'Seattle Kraken'
]

# # teams_df.index.values

for team in nhl_teams:

    if team not in team_check:
        print(team)

# Build opponent FPAPG

In [220]:
opponent_stats = skaters.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum'
    # Add more columns as needed
})

opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['date'], 2)
opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
# opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

opponent_stats = opponent_stats.rename_axis('abbreviation')

opponent_stats = pd.merge(opponent_stats, team_names, on='abbreviation')

opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats

,abbreviation,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,date,FPAPG,FPAPG_Rank,name,id,powerPlayPoints
0,ANA,177245,59,30,324,135,129,247.4,10,24.74,19,Anaheim Ducks,24,36
1,ARI,176822,46,27,315,226,144,240.1,10,24.01,22,Arizona Coyotes,53,28
2,BOS,179094,29,16,300,204,159,192.4,10,19.24,31,Boston Bruins,6,3
3,BUF,177137,48,29,313,211,147,239.9,10,23.99,23,Buffalo Sabres,7,12
4,CAR,195199,63,39,287,167,208,307.9,11,27.99,6,Carolina Hurricanes,12,29
5,CBJ,177997,51,32,324,116,181,258.0,10,25.80,14,Columbus Blue Jackets,29,15
6,CGY,176747,63,37,293,149,163,269.7,10,26.97,8,Calgary Flames,20,9
7,CHI,158981,57,32,308,151,146,248.4,9,27.60,7,Chicago Blackhawks,16,12
8,COL,159656,31,21,258,138,147,189.1,9,21.01,29,Colorado Avalanche,21,6
9,DAL,162065,38,23,311,177,138,208.3,9,23.14,24,Dallas Stars,25,6


# Add opponent FPAPG by position

In [221]:
skaters_temp = skaters
# replace_dict = {'LW': 'W', 'RW': 'W'}
skaters_temp['position'] = skaters_temp['position'].replace('LW', 'W')
skaters_temp['position'] = skaters_temp['position'].replace('RW', 'W')
skaters_temp = skaters_temp.loc[skaters_temp['gamePlayed']>0]


opponent_stats_by_pos = skaters_temp.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
    'team': 'unique'
    # Add more columns as needed
})
opponent_stats_by_pos

opponent_stats_by_pos['powerPlayPoints'] = opponent_stats_by_pos['powerPlayGoals'] + opponent_stats_by_pos['powerPlayAssists']

opponent_stats_by_pos = opponent_stats_by_pos.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats_by_pos = opponent_stats_by_pos.rename(columns={'date': 'gamesPlayed'})
opponent_stats_by_pos

D = opponent_stats_by_pos.xs('D', level='position')
D['position'] = 'D'
D['FPAPG'] = round(D['fantasyPoints'] / D['gamesPlayed'], 2)
D['FPAPG_Rank'] = D['FPAPG'].rank(ascending=False, method='min').astype(int)

C = opponent_stats_by_pos.xs('C', level='position')
C['position'] = 'C'
C['FPAPG'] = round(C['fantasyPoints'] / C['gamesPlayed'], 2)
C['FPAPG_Rank'] = C['FPAPG'].rank(ascending=False, method='min').astype(int)

W = opponent_stats_by_pos.xs('W', level='position')
W['position'] = 'W'
W['FPAPG'] = round(W['fantasyPoints'] / W['gamesPlayed'], 2)
W['FPAPG_Rank'] = W['FPAPG'].rank(ascending=False, method='min').astype(int)

skaters_temp['position'] = skaters_temp['position'].replace('W', 'F')
skaters_temp['position'] = skaters_temp['position'].replace('C', 'F')
opponent_stats_by_pos_f = skaters_temp.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
    'team': 'unique'
    # Add more columns as needed
})
opponent_stats_by_pos_f

opponent_stats_by_pos_f['powerPlayPoints'] = opponent_stats_by_pos_f['powerPlayGoals'] + opponent_stats_by_pos_f['powerPlayAssists']

opponent_stats_by_pos_f = opponent_stats_by_pos_f.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats_by_pos_f = opponent_stats_by_pos_f.rename(columns={'date': 'gamesPlayed'})

F = opponent_stats_by_pos_f.xs('F', level='position')
F['position'] = 'F'
F['FPAPG'] = round(F['fantasyPoints'] / F['gamesPlayed'], 2)
F['FPAPG_Rank'] = F['FPAPG'].rank(ascending=False, method='min').astype(int)

F

,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,gamesPlayed,team,powerPlayPoints,position,FPAPG,FPAPG_Rank
opponent,,,,,,,,,,,,,
ANA,108995,34,26,235,88,51,155.8,10,"[VGK, CAR, DAL, ARI, BOS, CBJ, PHI, PIT]",24,F,15.58,19
ARI,107492,30,23,231,148,54,151.9,10,"[NJD, NYR, NYI, STL, ANA, LAK, CHI, MTL]",22,F,15.19,21
BOS,110758,21,15,221,117,52,111.8,10,"[CHI, NSH, SJS, LAK, ANA, DET, FLA, TOR]",2,F,11.18,31
BUF,108145,31,25,237,152,45,148.9,10,"[NYR, NYI, TBL, CGY, MTL, OTT, NJD, COL, PHI]",10,F,14.89,23
CAR,119536,45,34,219,95,103,209.9,11,"[OTT, LAK, ANA, SJS, SEA, COL, TBL, PHI, NYR]",23,F,19.08,4
CBJ,107912,35,26,241,76,69,159.2,10,"[PHI, NYR, DET, CGY, MIN, ANA, MTL, NYI, DAL, ...",10,F,15.92,18
CGY,106789,41,29,214,88,56,162.2,10,"[WPG, PIT, WSH, BUF, CBJ, DET, NYR, STL, EDM, ...",6,F,16.22,15
CHI,96345,43,28,226,105,44,161.1,9,"[PIT, BOS, MTL, TOR, COL, VGK, ARI]",10,F,17.90,7
COL,98051,22,18,196,78,60,116.9,9,"[LAK, SJS, SEA, CHI, CAR, NYI, PIT, BUF, STL]",3,F,12.99,30


# Opponent goalie stats by FPAPG

In [222]:
gopponent_stats = goalies.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'goals_against': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
#     'team': 'unique',
    'powerPlaySaves': 'sum',
    'powerPlayShotsAgainst': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['date'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,timeOnIce,goals_against,shots,fantasyPoints,date,powerPlaySaves,powerPlayShotsAgainst,FPAPG,FPAPG_Rank,name
0,ANA,36135,31.0,279,6.6,10,41,47,0.66,26,Anaheim Ducks
1,ARI,36174,33.0,297,10.8,10,44,54,1.08,22,Arizona Coyotes
2,BOS,35675,28.0,317,7.8,10,50,55,0.78,24,Boston Bruins
3,BUF,35840,29.0,282,13.6,10,43,46,1.36,20,Buffalo Sabres
4,CAR,40037,37.0,377,19.0,11,57,68,1.73,18,Carolina Hurricanes
5,CBJ,36092,25.0,318,39.6,10,57,62,3.96,7,Columbus Blue Jackets
6,CGY,36081,21.0,327,54.2,10,54,60,5.42,2,Calgary Flames
7,CHI,32130,17.0,235,40.6,9,34,37,4.51,4,Chicago Blackhawks
8,COL,32374,27.0,302,16.0,9,46,53,1.78,17,Colorado Avalanche
9,DAL,32574,28.0,264,1.2,9,25,28,0.13,28,Dallas Stars


# Build module for Last five FP

### This is built off last five games for the PLAYER, not for the TEAM

In [223]:

last_five_df = pd.DataFrame()

for player in player_list:
    temp_df = all_df.loc[all_df['name'] == player]
    temp_df = temp_df.sort_values('date', ascending=False)
    temp_df = temp_df.head(5)
    last_five_df = pd.concat([last_five_df, temp_df])
    
last_five_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,...,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8473575-02-11-2023,3600,0,0,32,0,0,0,0,NaN,0,...,100.0,100.0,1,1.0,13.4,NaN,32.0,0.0,4.0,1.0
ID8473575-28-10-2023,3600,0,0,34,0,0,0,0,NaN,0,...,100.0,100.0,1,1.0,13.8,NaN,34.0,0.0,4.0,1.0
ID8473575-21-10-2023,3547,0,0,42,0,0,0,0,NaN,0,...,100.0,90.909091,0,1.0,1.8,NaN,42.0,3.0,0.0,0.0
ID8480280-02-11-2023,3900,0,0,35,0,0,0,0,NaN,0,...,100.0,93.333333,0,1.0,6.6,NaN,35.0,2.0,4.0,1.0
ID8480280-28-10-2023,3600,0,0,24,0,0,0,0,NaN,0,...,100.0,94.736842,0,1.0,6.6,NaN,24.0,1.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8481611-21-10-2023,3417,0,0,27,0,0,0,0,NaN,0,...,50.0,88.888889,0,1.0,-7.8,NaN,27.0,6.0,0.0,0.0
ID8481611-19-10-2023,2200,0,0,17,0,0,0,0,NaN,0,...,NaN,88.235294,0,1.0,-1.0,NaN,17.0,2.0,0.0,0.0
ID8481035-01-11-2023,2956,0,0,9,0,0,0,0,NaN,0,...,NaN,75.0,0,1.0,-2.6,NaN,9.0,2.0,0.0,0.0
ID8481035-28-10-2023,3572,0,0,25,0,0,0,0,NaN,0,...,50.0,78.947368,0,1.0,-10.4,NaN,25.0,7.0,0.0,0.0


### Let's build one for team's last five games

In [224]:
team_names

last_five_df_team = pd.DataFrame()

for index, row in team_names.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('date', ascending=False)
    five_dates = temp_all['date'].unique()[:5]
    temp_filtered = temp_all[temp_all['date'].isin(five_dates)]
    last_five_df_team = pd.concat([last_five_df_team, temp_filtered])
    
last_five_df_team

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,...,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8482110-02-11-2023,950,0,0,2,0,0,0,0,NaN,0,...,NaN,NaN,0,1.0,0.7,0.0,NaN,NaN,NaN,NaN
ID8477970-02-11-2023,3600,0,0,25,0,0,0,0,NaN,0,...,NaN,90.47619,0,1.0,2.4,NaN,25.0,3.0,4.0,1.0
ID8479407-02-11-2023,1058,3,1,2,2,1,2,0,0.0,0,...,NaN,NaN,0,1.0,6.9,3.0,NaN,NaN,NaN,NaN
ID8477508-02-11-2023,847,0,0,2,0,0,0,0,100.0,3,...,NaN,NaN,0,1.0,1.7,0.0,NaN,NaN,NaN,NaN
ID8479414-02-11-2023,504,1,0,1,2,0,0,2,NaN,0,...,NaN,NaN,0,1.0,1.8,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8477505-24-10-2023,1411,1,0,1,0,0,0,2,61.54,8,...,NaN,NaN,0,1.0,2.1,0.0,NaN,NaN,NaN,NaN
ID8478407-24-10-2023,1512,2,0,0,0,0,1,0,NaN,0,...,NaN,NaN,0,1.0,2.5,1.0,NaN,NaN,NaN,NaN
ID8479977-24-10-2023,419,0,0,1,0,0,0,0,100.0,1,...,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8475768-24-10-2023,1024,0,2,5,0,1,0,0,60.0,3,...,NaN,NaN,0,1.0,5.0,1.0,NaN,NaN,NaN,NaN


In [225]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last5 = last_five_df_team.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last5['gamePlayed'] = summary_stats_last5['gamePlayed'].astype(int)
summary_stats_last5['decisionPoints'] = summary_stats_last5['decisionPoints'].astype(int)
summary_stats_last5['wins'] = summary_stats_last5['wins'].astype(int)
summary_stats_last5['specialTeams'] = summary_stats_last5['specialTeams'].astype(int)


summary_stats_last5['FPP60'] = (summary_stats_last5['fantasyPoints'] / summary_stats_last5['timeOnIce'] * 3600).round(2)
summary_stats_last5['FPPG'] = (summary_stats_last5['fantasyPoints'] / summary_stats_last5['gamePlayed']).round(2)

summary_stats_last5 = summary_stats_last5.sort_values(by='FPPG', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats_last5.set_index('name', inplace=True)

player_list = summary_stats.index.to_list()



summary_stats_last5 = summary_stats_last5.merge(rostership, on='name', how='left')

summary_stats_last5.set_index('name', inplace=True)

summary_stats_last5.head()

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,...,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,rostered
name,,,,,,,,,,,,,,,,,,,,,
Semyon Varlamov,ID8473575,NYI,G,7200,2,0,0,0,66,0,...,2,55,9,8,2,2,27.2,13.60,13.60,3.297906
Jonathan Quick,ID8471734,NYR,G,3585,1,0,0,0,29,0,...,1,27,1,4,1,1,12.8,12.85,12.80,7.526228
Joel Hofer,ID8480981,STL,G,3596,1,0,0,0,27,0,...,0,26,1,4,1,1,12.4,12.41,12.40,0.682755
Antti Raanta,ID8477293,CAR,G,3600,1,0,0,0,20,0,...,1,16,3,4,1,1,11.0,11.00,11.00,4.473232
Jeremy Swayman,ID8480280,BOS,G,11089,3,0,0,0,82,0,...,4,70,8,12,3,1,24.8,8.05,8.27,91.198115


In [226]:
summary_stats_last5_avail = summary_stats_last5.loc[summary_stats_last5['rostered'] <= 60]

# summary_stats_last5_avail_effective = summary_stats_last5_avail.loc[summary_stats_last5_avail['FPPG'] >= 1.5]

In [227]:
summary_stats_last5_avail = summary_stats_last5_avail.loc[summary_stats_last5_avail['position'] != 'G']

top_players_by_team = summary_stats_last5_avail.groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)

top_players_by_team = top_players_by_team.loc[top_players_by_team['FPPG'] >= 1.7]

top_players_by_team.loc[top_players_by_team['team'] == 'EDM']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,...,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,rostered
name,,,,,,,,,,,,,,,,,,,,,
Evander Kane,ID8475169,EDM,LW,5789,5,3,4,1,19,23,...,0,0,0,0,0,0,15.2,9.45,3.04,46.615046
Sam Gagner,ID8474040,EDM,C,780,1,2,0,0,5,3,...,0,0,0,0,0,0,4.8,22.15,4.80,0.059212


In [228]:
goalies_summary_last_5 = summary_stats_last5.loc[summary_stats_last5['position'] == "G"]
goalies_summary_last_5 = goalies_summary_last_5.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])
goalies_summary_last_5

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,rostered
name,,,,,,,,,,,,,,,,,,,,
Semyon Varlamov,ID8473575,NYI,G,7200,2,66,66,9,2,55,2,55,9,8,2,2,27.2,13.60,13.60,3.297906
Jonathan Quick,ID8471734,NYR,G,3585,1,29,29,1,1,27,1,27,1,4,1,1,12.8,12.85,12.80,7.526228
Joel Hofer,ID8480981,STL,G,3596,1,27,27,1,0,26,0,26,1,4,1,1,12.4,12.41,12.40,0.682755
Antti Raanta,ID8477293,CAR,G,3600,1,20,20,3,1,16,1,16,3,4,1,1,11.0,11.00,11.00,4.473232
Jeremy Swayman,ID8480280,BOS,G,11089,3,82,79,8,4,67,4,70,8,12,3,1,24.8,8.05,8.27,91.198115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samuel Ersson,ID8481035,PHI,G,6528,2,34,25,3,1,21,2,27,5,0,0,0,-13.0,-7.17,-6.50,0.182457
Jack Campbell,ID8475789,EDM,G,3557,1,31,25,7,1,17,1,23,7,0,0,0,-7.0,-7.08,-7.00,8.211304
Dan Vladar,ID8478435,CGY,G,3591,1,30,24,9,0,15,0,21,9,0,0,0,-7.2,-7.22,-7.20,0.307164


# Create a DataFrame of the sked

In [229]:
# Create a list to store game information
game_info = []

# Iterate through the dates
for date_info in days[1:4]:
        
    games = date_info['games']
    for game in games:
        
        date = game['gameDate']

        datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

        utc_timestamp = pytz.utc.localize(datetime_obj)
        eastern_timezone = pytz.timezone('US/Eastern')
        eastern_time = utc_timestamp.astimezone(eastern_timezone)

        formatted_date_string = eastern_time.strftime('%d-%m-%Y')
        
        game_info.append({
            'Game ID': game['gamePk'],
            'Date': formatted_date_string,
            'Home Team': game['teams']['home']['team']['name'],
            'Home': team_names.loc[team_names['name'] == game['teams']['home']['team']['name']]['abbreviation'].iloc[0],
#             'HomeQB': opponent_QBs.loc[opponent_QBs['name'] == game['teams']['home']['team']['name']]['FPAPG'].iloc[0],
            'Away Team': game['teams']['away']['team']['name'],
            'Away': team_names.loc[team_names['name'] == game['teams']['away']['team']['name']]['abbreviation'].iloc[0],
#             'AwayQB': opponent_QBs.loc[opponent_QBs['name'] == game['teams']['away']['team']['name']]['FPAPG'].iloc[0],
        })

# Create a DataFrame
df = pd.DataFrame(game_info)


df.rename(columns={'Home Team': 'focusTeam', 'Date': 'date', 'Home': 'focus'}, inplace=True)
df_temp = df[['Game ID', 'date', 'Away Team', 'Away', 'focusTeam', 'focus']]
df_temp.rename(columns={'focusTeam': 'opponent', 'Date': 'date', 'focus': 'opp', 'Away Team': 'focusTeam', 'Away': 'focus'}, inplace=True)
df.rename(columns={'Away Team': 'opponent', 'Away': 'opp'}, inplace=True)
df['H/A'] = 'H'
df_temp['H/A'] = 'A'
df = pd.concat([df, df_temp])
df

sked_stats = df
sked_stats.reset_index(inplace=True)

In [230]:
# sked_stats['Dforecast'] = D.loc[]



for index, row in sked_stats.iterrows():
    d = D.loc[D.index == row['opp']]['FPAPG'][0]
#     drank = D.loc[D.index == row['focus']]['FPAPG_Rank'][0]
    c = C.loc[C.index == row['opp']]['FPAPG'][0]
#     crank = C.loc[C.index == row['focus']]['FPAPG_Rank'][0]
    w = W.loc[W.index == row['opp']]['FPAPG'][0]
#     wrank = W.loc[W.index == row['focus']]['FPAPG_Rank'][0]
    g = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['opp']]['FPAPG'].iloc[0]
    
    sked_stats.at[index, 'D_FPAPG'] = d
#     sked_stats.at[index, 'D_RANK'] = drank
    sked_stats.at[index, 'C_FPAPG'] = c
#     sked_stats.at[index, 'C_RANK'] = crank
    sked_stats.at[index, 'W_FPAPG'] = w
#     sked_stats.at[index, 'W_RANK'] = wrank
    sked_stats.at[index, 'G_FPAPG'] = g
    
sked_stats.groupby('focusTeam').sum().sort_values('D_FPAPG', ascending=False)

,index,Game ID,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG
focusTeam,,,,,,
Philadelphia Flyers,9,4046040320,19.79,13.58,16.75,0.92
Chicago Blackhawks,28,4046040342,19.48,16.86,16.69,3.08
St. Louis Blues,5,4046040322,18.31,19.25,16.10,1.14
Buffalo Sabres,7,4046040322,18.07,15.35,17.47,4.68
New Jersey Devils,18,4046040332,17.42,16.89,14.12,8.94
Vegas Golden Knights,34,4046040348,17.18,13.93,14.64,2.44
Seattle Kraken,13,2023020170,10.75,6.78,9.44,5.42
Pittsburgh Penguins,14,2023020171,10.67,10.18,10.04,7.44
Calgary Flames,13,2023020170,10.53,9.16,6.76,3.96


In [231]:
sked_stats

,index,Game ID,date,focusTeam,focus,opponent,opp,H/A,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG
0,0,2023020157,03-11-2023,Buffalo Sabres,BUF,Philadelphia Flyers,PHI,H,8.44,7.49,8.83,2.42
1,1,2023020158,03-11-2023,St. Louis Blues,STL,New Jersey Devils,NJD,H,10.96,9.03,8.29,-0.76
2,2,2023020159,04-11-2023,Edmonton Oilers,EDM,Nashville Predators,NSH,H,9.44,6.39,8.77,3.00
3,3,2023020160,04-11-2023,Arizona Coyotes,ARI,Winnipeg Jets,WPG,H,10.42,9.67,8.16,2.86
4,4,2023020164,04-11-2023,St. Louis Blues,STL,Montréal Canadiens,MTL,H,7.35,10.22,7.81,1.90
5,5,2023020161,04-11-2023,Detroit Red Wings,DET,Boston Bruins,BOS,H,8.06,6.94,4.24,0.78
6,6,2023020162,04-11-2023,Ottawa Senators,OTT,Tampa Bay Lightning,TBL,H,8.63,7.36,8.73,1.32
7,7,2023020165,04-11-2023,Toronto Maple Leafs,TOR,Buffalo Sabres,BUF,H,9.10,6.57,8.32,1.36
8,8,2023020166,04-11-2023,Washington Capitals,WSH,Columbus Blue Jackets,CBJ,H,9.88,7.52,8.40,3.96
9,9,2023020163,04-11-2023,Philadelphia Flyers,PHI,Los Angeles Kings,LAK,H,10.69,7.01,8.43,-0.44


# Run the printout module

In [232]:
eastern = pytz.timezone('US/Eastern')

for day in days[1:4]:
    date = day['date']
    games = day['games']

    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)

        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')

        home_abbr = opponent_stats.loc[opponent_stats['name'] == home_team, 'abbreviation'].values[0]
        home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
        home_goalies = goalies_summary_last_5.loc[goalies_summary_last_5['team'] == home_abbr]

        away_abbr = opponent_stats.loc[opponent_stats['name'] == away_team, 'abbreviation'].values[0]
        away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
        away_goalies = goalies_summary_last_5.loc[goalies_summary_last_5['team'] == away_abbr]

#         print(f"On {date}, {away_team} at {home_team}")
        print(f"\033[1m{away_team} at {home_team}\033[0m")
        print(f"{formatted_time}, {venue}")
        print()
        print(f"\033[1mForwards against {home_abbr} score\033[0m: {F.loc[F.index == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {F.loc[F.index == home_abbr, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mDefence against {home_abbr} score\033[0m: {D.loc[D.index == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {D.loc[D.index == home_abbr, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop {away_abbr} Players Fantasy Points Last 5 (60% or fewer rosters)\033[0m")
        for index, row in away_top5.iterrows():
            print(f"{index}, {row['position']} ({round(row['rostered'], 2)}% rostered): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
        print()
        print(f"\033[1mGoalies against {home_abbr} score\033[0m: {gopponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
        for index, row in away_goalies.iterrows():
            print(f"{index} ({round(row['rostered'], 2)}% rostered) last five: {row['gamePlayed']} appearances, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")
        print()
        print(f"\033[1mForwards against {away_abbr} score\033[0m: {F.loc[F.index == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {F.loc[F.index == away_abbr, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mDefence against {away_abbr} score\033[0m: {D.loc[D.index == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {D.loc[D.index == away_abbr, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop {home_abbr} Players Fantasy Points Last 5 (60% or fewer rosters)\033[0m")
        for index, row in home_top5.iterrows():
            print(f"{index}, {row['position']} ({round(row['rostered'], 2)}% rostered): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")

        print()
        print(f"\033[1mGoalies against {away_abbr} score\033[0m: {gopponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
        for index, row in home_goalies.iterrows():
            print(f"{index} ({round(row['rostered'], 2)}% rostered) last five: {row['gamePlayed']} appearances, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")
        print()
        print('-'*25)
        print()
#         print("\033[1mThis is bold text\033[0m")


FRIDAY
Philadelphia Flyers at Buffalo Sabres
7 p.m., KeyBank Center

Forwards against BUF score: 14.89 FPAPG, Rank: 23
Defence against BUF score: 9.1 FPAPG, Rank: 17
Top PHI Players Fantasy Points Last 5 (60% or fewer rosters)
Travis Sanheim, D (52.99% rostered): 13.3 fantasy points (2.66 FPPG)
Cam Atkinson, RW (27.6% rostered): 13.2 fantasy points (2.64 FPPG)
Joel Farabee, LW (4.07% rostered): 8.8 fantasy points (1.76 FPPG)
Bobby Brink, RW (2.45% rostered): 8.6 fantasy points (1.72 FPPG)

Goalies against BUF score: 1.36 FPAPG, Rank: 20
Carter Hart (80.66% rostered) last five: 4 appearances, FPPG: 0.2, FP: 0.8
Samuel Ersson (0.18% rostered) last five: 2 appearances, FPPG: -6.5, FP: -13.0

Forwards against PHI score: 16.32 FPAPG, Rank: 13
Defence against PHI score: 8.44 FPAPG, Rank: 23
Top BUF Players Fantasy Points Last 5 (60% or fewer rosters)
Mattias Samuelsson, D (14.42% rostered): 11.7 fantasy points (2.34 FPPG)
Casey Mittelstadt, C (6.34% rostered): 9.8 fantasy points (1.96 FPPG)


In [244]:
def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"

convert_to_minutes_seconds(summary_stats.loc[summary_stats['name'] == "Barrett Hayton"]['timeOnIce'].values[0])


'195:05'